## Loading the dataset and training an image classifier

In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
import torch
torch.cuda.is_available()

True

In [4]:
import cv2
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

In [22]:
# Defining the paths

path_data = os.path.join(os.path.dirname(os.getcwd()),"Data")
path_images = os.path.join(path_data,"images\\images")
path_images

'd:\\Study\\Conda Projects\\opdl\\Data\\images\\images'

In [8]:
path_labels = os.path.join(path_data,"annotations.csv")
path_labels

'd:\\Study\\Conda Projects\\opdl\\Data\\annotations.csv'

### Loading the dataset using custom loader

In [27]:
# Loading the dataset

class dogCatDataset(Dataset):

    def __init__(self, csv_file, root_dir, transform=None):
        self.labels = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, index):
        if torch.is_tensor(index):
            index = index.to_list()
        
        img_name = os.path.join(self.root_dir,self.labels.iloc[index,0])
        img_name = img_name +".jpg"
        image = cv2.imread(img_name)
        label = self.labels.iloc[index,1]

        if self.transform:
            image = self.transform(image)
        
        return image,label

In [38]:
# Defining the transform

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224,224))
])

In [39]:
# Creating the dataset
dataset_cd = dogCatDataset(csv_file=path_labels, root_dir=path_images, transform=transform)
dataloader_cd = DataLoader(dataset_cd, batch_size=32, shuffle=True)

In [40]:
len(dataset_cd)

3680

In [41]:
for i, sample in enumerate(dataset_cd):
    print(i, sample[0].size())

    if i == 3:
        break

0 torch.Size([3, 224, 224])
1 torch.Size([3, 224, 224])
2 torch.Size([3, 224, 224])
3 torch.Size([3, 224, 224])


### Building the neural network